# **Вебинар 10. HTTP-запросы и веб-взаимодействие**

## Проверка связи
Если у вас нет звука:
*   убедитесь, что на вашем устройстве и в колонках включён звук
*   обновите страницу вебинара или закройте страницу и заново присоединитесь к вебинару
*   откройте вебинар в другом браузере
*   перезагрузите ваше устройство и попытайтесь войти снова

Поставьте в чат:
* «+» — если видно и слышно
* «–» — если нет
## О спикере
**Погудина Дарья**
- эксперт-разработчик на Python в области информационной безопасности и в финтехе
- сертифицированный специалист по базам данных
- более 8 лет опыта разработки, из них более 4 лет — на Python
- 3 года в сфере информационной безопасности с разработкой ПО на Python
## Правила участия
*   Приготовьте блокнот и ручку, чтобы записывать важные мысли и идеи
*   Продолжительность вебинара — 1 час 20 минут
*   Вы можете писать свои вопросы в чате или задавать их вслух
*   Запись вебинара будет доступна в личном кабинете


## Связь с предыдущей темой

На прошлом занятии мы изучали **API** — как приложения обмениваются данными через программные интерфейсы. Сегодня углубимся в **HTTP-протокол** — фундамент, на котором работают все API и веб-приложения.

**Что мы уже знаем:**
- Как делать запросы к API через `requests`.
- REST API принципы.
- Авторизация и безопасность API.

**Что изучим сегодня:**
- Детали HTTP-протокола
- Современные инструменты (`httpx` для асинхронности)
- Парсинг HTML-страниц
- Анализ сетевого трафика для кибербезопасности
- Практические навыки для пентестов

## План занятия ##
1. HTTP: протокол веб-взаимодействия
2. Библиотеки для HTTP-запросов
3. Разбор HTML с помощью BeautifulSoup
4. Безопасность HTTP-запросов
5. Работа с JSON API
6. Scapy: анализ сетевых пакетов для кибербезопасности

---

# **1. HTTP: протокол веб-взаимодействия**

### **Что такое HTTP**

**HTTP (hypertext transfer protocol)** — протокол прикладного уровня, который определяет формат обмена данными между клиентом и сервером.

**Клиент-серверная архитектура:**

```
Клиент (браузер/скрипт)  →  HTTP-запрос  →  Сервер
Клиент                   ←  HTTP-ответ   ←  Сервер
```

**Основные компоненты HTTP-запроса:**

1. **Метод** (GET, POST, PUT, DELETE и др.).
2. **URL** (адрес ресурса).
3. **Заголовки** (метаданные: User-Agent, Authorization, Content-Type).
4. **Тело запроса** (для POST/PUT — данные).

**Основные компоненты HTTP-ответа:**

1. **Статус-код** (200, 404, 500 и др.).
2. **Заголовки** (Content-Type, Set-Cookie и др.).
3. **Тело ответа** (HTML, JSON, изображение и др.).

### HTTP vs HTTPS

**HTTP** — незащищённый протокол:
- ❌ данные передаются в открытом виде;
- ❌ легко перехватить трафик;
- ❌ уязвим для MitM атак.

**HTTPS** — защищённый протокол:
- ✅ данные шифруются (TLS/SSL);
- ✅ защита от перехвата;
- ✅ проверка подлинности сервера.

---

## Ваши вопросы

# **2. Библиотеки для HTTP-запросов**

### **Requests vs httpx**

**`requests`** — классическая библиотека:
- ✅ простая и надёжная;
- ✅ синхронная (один запрос за раз);
- ✅ широко используется;
- ❌ нет асинхронности;
- ❌ нет HTTP/2.

**`httpx`** — современная альтернатива:
- ✅ асинхронность (множество запросов одновременно);
- ✅ HTTP/2 поддержка;
- ✅ встроенные таймауты;
- ✅ гибкая работа с прокси;
- ✅ совместим с `requests` API.

**Возможности `httpx` для кибербезопасности:**
- сканирование множества эндпоинтов одновременно;
- массовые запросы при тестировании;
- параллельная проверка уязвимостей.

---

## **Requests — классический подход**

### Установка

```bash
pip install requests
```

### Простой GET-запрос

```python
import requests

response = requests.get("https://example.com")
print(f"Status: {response.status_code}")
print(f"Content-Type: {response.headers.get('Content-Type')}")
print(response.text[:500])  # Первые 500 символов
```

Основные атрибуты ответа:
- `response.status_code` — HTTP-код (200, 404, 500);
- `response.text` — содержимое как строка;
- `response.json()` — парсинг JSON (если возможно);
- `response.headers` — заголовки ответа;
- `response.cookies` — cookies.

### POST-запрос с данными

```python
# Форма (application/x-www-form-urlencoded)
data = {"username": "admin", "password": "secret"}
response = requests.post("https://httpbin.org/post", data=data)

# JSON (application/json)
json_data = {"name": "Alex", "age": 25}
response = requests.post("https://httpbin.org/post", json=json_data)
```

### Заголовки и параметры

```python
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json"
}
params = {"page": 1, "limit": 10}

response = requests.get(
    "https://api.example.com/data",
    headers=headers,
    params=params
)
```

### Обработка ошибок

```python
import requests
from requests.exceptions import Timeout, ConnectionError, HTTPError

try:
    response = requests.get("https://example.com", timeout=5)
    response.raise_for_status()  # Вызовет исключение при 4xx/5xx
    data = response.json()
except Timeout:
    print("Превышено время ожидания")
except ConnectionError:
    print("Ошибка соединения")
except HTTPError as e:
    print(f"HTTP ошибка: {e}")
except ValueError:
    print("Не удалось распарсить JSON")
```

## **httpx — современный инструмент для кибербезопасности**

### **Зачем нужен httpx**

В кибербезопасности часто нужно:
- сканировать множество URL одновременно;
- тестировать на уязвимости параллельно;
- делать массовые запросы при пентестах;
- работать с HTTP/2 для анализа современных протоколов.

### Установка

In [ ]:
pip install httpx

### Синхронное использование (как requests)

In [ ]:
import httpx

# Синхронный запрос (похож на requests)
response = httpx.get("https://example.com", timeout=10.0)
print(response.status_code)
print(response.text)

**Преимущества даже в синхронном режиме:**
- встроенные таймауты (**обязательны**);
- лучшая работа с прокси;
- HTTP/2 поддержка.

### Асинхронное использование (главное преимущество)

In [ ]:
import httpx
import asyncio

async def fetch_url(url):
    async with httpx.AsyncClient(timeout=10.0) as client:
        response = await client.get(url)
        return {
            "url": url,
            "status": response.status_code,
            "content_type": response.headers.get("Content-Type")
        }

async def scan_multiple_urls(urls):
    tasks = [fetch_url(url) for url in urls]
    results = await asyncio.gather(*tasks)
    return results

# Использование
urls = [
    "https://example.com",
    "https://httpbin.org/get",
    "https://api.github.com"
]

results = await scan_multiple_urls(urls)
for result in results:
    print(f"{result['url']}: {result['status']}")

**Сравнение скорости:**

In [ ]:
import time
import requests
import httpx
import asyncio

urls = [f"https://httpbin.org/delay/1" for _ in range(10)]

# Синхронно (requests) — последовательно
start = time.time()
for url in urls:
    requests.get(url)
print(f"Requests (sync): {time.time() - start:.2f} сек")

# Асинхронно (httpx) — параллельно
async def async_fetch():
    async with httpx.AsyncClient() as client:
        tasks = [client.get(url) for url in urls]
        await asyncio.gather(*tasks)

start = time.time()
await async_fetch()
print(f"httpx (async): {time.time() - start:.2f} сек")

### Практический пример: сканер уязвимостей

In [ ]:
import httpx
import asyncio
from urllib.parse import urljoin

async def check_endpoint(base_url, endpoint):
    url = urljoin(base_url, endpoint)
    async with httpx.AsyncClient(timeout=5.0, follow_redirects=True) as client:
        try:
            response = await client.get(url)
            return {
                "url": url,
                "status": response.status_code,
                "exists": response.status_code < 400
            }
        except Exception as e:
            return {"url": url, "status": "error", "error": str(e)}

async def scan_directories(base_url, directories):
    tasks = [check_endpoint(base_url, dir_path) for dir_path in directories]
    results = await asyncio.gather(*tasks)
    return [r for r in results if r.get("exists")]

# Использование
base_url = "https://httpbin.org"
common_dirs = ["/admin", "/api", "/.git", "/backup", "/test"]

found = await  scan_directories(base_url, common_dirs)
for item in found:
    print(f"Найдено: {item['url']} ({item['status']})")

### HTTP/2 поддержка

In [ ]:
import httpx

# HTTP/2 автоматически, если сервер поддерживает
async with httpx.AsyncClient(http2=True) as client:
    response = await client.get("https://www.google.com")
    print(f"HTTP версия: {response.http_version}")  # HTTP/2.0

### Работа с прокси (гибче, чем requests)

In [ ]:
import httpx
import random

# Один прокси
proxy = "http://proxy.example.com:8080"
async with httpx.AsyncClient(proxy=proxy) as client:
    response = await client.get("https://example.com")

# Ротация прокси
proxy_list = [
    "http://proxy1.example.com:8080",
    "http://proxy2.example.com:8080",
    "http://proxy3.example.com:8080"
]

selected_proxy = random.choice(proxy_list)
async with httpx.AsyncClient(proxy=selected_proxy) as client:
    response = await client.get("https://example.com")

## **Сравнение requests и httpx**

| Характеристика | requests | httpx |
|----------------|----------|-------|
| **Синхронность** | ✅ Да | ✅ Да |
| **Асинхронность** | ❌ Нет | ✅ Да |
| **HTTP/2** | ❌ Нет | ✅ Да |
| **Таймауты** | Опциональны | Обязательны |
| **Прокси** | ✅ Да | ✅ Да (гибче) |
| **Скорость (1 запрос)** | Быстро | Быстро |
| **Скорость (много запросов)** | Медленно | Очень быстро |
| **Использование** | Простые задачи | Кибербезопасность, сканирование |

**Рекомендации:**
- **requests** — для простых задач, обучения
- **httpx** — для сканирования, пентестов, массовых запросов

## Ваши вопросы

# **3. Разбор HTML с помощью BeautifulSoup**

### **Зачем нужен парсинг HTML**

В кибербезопасности парсинг HTML используется для:
- разведки — извлечения информации с сайтов;
- поиска уязвимостей — анализа форм, ссылок, скриптов;
- автоматизации — взаимодействия с веб-интерфейсами;
- сбора данных — извлечения структурированной информации.

### Установка

In [ ]:
pip install beautifulsoup4 lxml

### Базовое использование

In [ ]:
from bs4 import BeautifulSoup
import requests

response = requests.get("https://httpbin.org")
soup = BeautifulSoup(response.text, "html.parser")

# Найти все ссылки
for link in soup.find_all("a"):
    href = link.get("href")
    text = link.text.strip()
    if href:
        print(f"{text}: {href}")

# Найти все формы
for form in soup.find_all("form"):
    action = form.get("action")
    method = form.get("method", "GET")
    print(f"Form: {method} {action}")

### Поиск элементов

In [ ]:
# По тегу
soup.find("div")  # Первый div
soup.find_all("div")  # Все div

# По классу
soup.find(class_="header")
soup.find_all(class_="menu-item")

# По ID
soup.find(id="main-content")

# По атрибуту
soup.find_all("input", type="password")  # Все поля паролей
soup.find_all("a", href=True)  # Все ссылки с href

# CSS-селекторы (мощный инструмент)
soup.select("div.content > p")  # Параграфы внутри div.content
soup.select("a[href^='https://']")  # Внешние ссылки

### Практический пример

In [ ]:
from bs4 import BeautifulSoup
import requests

def analyze_forms(url):
    """Анализ форм на странице для поиска уязвимостей"""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    forms = []
    for form in soup.find_all("form"):
        form_data = {
            "action": form.get("action", ""),
            "method": form.get("method", "GET").upper(),
            "inputs": []
        }

        # Анализ полей ввода
        for input_tag in form.find_all("input"):
            input_data = {
                "name": input_tag.get("name"),
                "type": input_tag.get("type", "text"),
                "required": input_tag.has_attr("required")
            }
            form_data["inputs"].append(input_data)

        # Поиск потенциальных уязвимостей
        if form_data["method"] == "GET":
            print(f"Форма использует GET для отправки данных: {form_data['action']}")

        if any(inp["type"] == "password" for inp in form_data["inputs"]):
            if not url.startswith("https://"):
                print(f"Attention: Пароль передаётся через HTTP!")

        forms.append(form_data)

    return forms

# Использование
forms = analyze_forms("http://httpbin.org/forms/post")

Attention: Пароль передаётся через HTTP!


### Извлечение данных

In [ ]:
# Текст элемента
title = soup.find("h1").text.strip()

# Атрибуты
img_src = soup.find("img").get("src")
link_href = soup.find("a").get("href")

# Вложенные элементы
div = soup.find("div", class_="content")
paragraphs = div.find_all("p")
text = " ".join([p.text for p in paragraphs])

### Поиск скриптов и метаданных

In [ ]:
# Все скрипты (может содержать чувствительную информацию)
for script in soup.find_all("script"):
    if script.string:
        # Поиск API ключей, токенов в JavaScript
        if "api_key" in script.string.lower():
            print("Возможная утечка API ключа в JavaScript!")

# Мета-теги
for meta in soup.find_all("meta"):
    name = meta.get("name") or meta.get("property")
    content = meta.get("content")
    print(f"{name}: {content}")

## **Пример парсера для разведки**

In [ ]:
from bs4 import BeautifulSoup
import requests
import re

def extract_site_info(url):
    """Извлечение информации о сайте для разведки"""
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")

        info = {
            "title": soup.find("title").text if soup.find("title") else None,
            "links": [],
            "emails": [],
            "forms": [],
            "scripts": []
        }

        # Все ссылки
        for link in soup.find_all("a", href=True):
            href = link.get("href")
            if href.startswith("http"):
                info["links"].append(href)

        # Email-адреса
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        emails = re.findall(email_pattern, response.text)
        info["emails"] = list(set(emails))

        # Формы
        for form in soup.find_all("form"):
            info["forms"].append({
                "action": form.get("action"),
                "method": form.get("method", "GET")
            })

        # Внешние скрипты
        for script in soup.find_all("script", src=True):
            info["scripts"].append(script.get("src"))

        return info
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

# Использование
info = extract_site_info("https://example.com")
if info:
    print(f"Заголовок: {info['title']}")
    print(f"Найдено ссылок: {len(info['links'])}")
    print(f"Найдено email: {info['emails']}")
    print(f"Найдено форм: {len(info['forms'])}")

## Ваши вопросы

# **4. Безопасность HTTP-запросов**

### **Критические правила**

**1. Всегда используйте HTTPS**
```python
# ✅ Правильно
url = "https://example.com"

# ❌ Никогда
url = "http://example.com"  # Данные в открытом виде
```

**2. Никогда не храните секреты в коде**
```python
# ❌ Критическая ошибка
password = "secret123"
api_key = "sk_live_1234567890"

# ✅ Правильно
import os
password = os.getenv("PASSWORD")
api_key = os.getenv("API_KEY")
```

**3. Всегда устанавливайте таймауты**
```python
# ✅ Правильно
response = requests.get(url, timeout=10)

# ❌ ОПАСНО (может зависнуть навсегда)
response = requests.get(url)
```

**4. Проверяйте сертификаты SSL**
```python
# ✅ По умолчанию проверяются
response = requests.get("https://example.com")

# ❌ Опасно (отключение проверки)
response = requests.get("https://example.com", verify=False)
# Использовать только для тестирования собственных серверов
```

**5. Валидация входных данных**
```python
import re
from urllib.parse import urlparse

def is_valid_url(url):
    """Проверка корректности URL"""
    try:
        result = urlparse(url)
        return all([result.scheme in ['http', 'https'], result.netloc])
    except:
        return False

# Использование
url = input("Введите URL: ")
if not is_valid_url(url):
    raise ValueError("Неверный формат URL")
```

### Защита от утечек данных

```python
import logging

# Настройка логирования (НЕ логировать секреты)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def safe_request(url, api_key=None):
    """Безопасный запрос с защитой от утечек"""
    headers = {}
    if api_key:
        headers["Authorization"] = f"Bearer {api_key}"
    
    logger.info(f"Запрос к {url}")
    
    response = requests.get(url, headers=headers, timeout=10)
    return response
```


## Ваши вопросы

# **5. Работа с JSON API**

Многие современные сервисы предоставляют данные в формате JSON (мы уже изучали это в теме про API).

```python
import requests

response = requests.get("https://api.github.com/users/octocat")
data = response.json()

print(f"Логин: {data['login']}")
print(f"Репозиториев: {data['public_repos']}")
```

**Обработка ошибок JSON:**
```python
try:
    data = response.json()
except ValueError:
    print("Ответ не является валидным JSON")
    print(f"Content-Type: {response.headers.get('Content-Type')}")
    print(f"Первые 200 символов: {response.text[:200]}")
```


## Ваши вопросы

# **6. Scapy: анализ сетевых пакетов для кибербезопасности**

### **Что такое Scapy**

`scapy` — мощный инструмент для:
- анализа сетевого трафика — перехват и изучение пакетов;
- генерации пакетов — создание кастомных сетевых пакетов;
- сетевого сканирования — обнаружение хостов и портов;
- тестирования безопасности — проверка сетевых защит.

### Установка

In [ ]:
pip install scapy

**⚠️ Важно:** Для работы требуются права администратора (root/sudo)!

### Базовое использование

In [ ]:
# Примечание: для реального перехвата (sniff) нужны права администратора
# В Jupyter notebook это обычно не работает
# Используйте scapy в обычном Python-скрипте с sudo

from scapy.all import sniff

# Перехват пакетов
packets = sniff(count=10)  # Перехватить 10 пакетов
packets.summary()  # Краткая информация

# Детальная информация о пакете
packet = packets[0]
packet.show()  # Полная структура пакета

### Анализ HTTP-трафика

In [ ]:
from scapy.all import sniff
from scapy.layers.http import HTTPRequest, HTTPResponse


def process_packet(packet):
    """Обработка HTTP пакетов"""
    if packet.haslayer(HTTPRequest):
        http_layer = packet[HTTPRequest]
        print(f"[HTTP REQUEST] {http_layer.Method} {http_layer.Host}{http_layer.Path}")

        # Извлечение заголовков
        if http_layer.Authorization:
            print(f"Обнаружена авторизация: {http_layer.Authorization[:50]}...")

        # Дополнительная информация
        if http_layer.User_Agent:
            print(f"User-Agent: {http_layer.User_Agent[:80]}")

    if packet.haslayer(HTTPResponse):
        http_layer = packet[HTTPResponse]
        print(f"[HTTP RESPONSE] Status: {http_layer.Status_Code}")

        # Размер ответа
        if hasattr(http_layer, 'Content_Length'):
            print(f"Content-Length: {http_layer.Content_Length}")

try:
    packets = sniff(filter="tcp port 80", prn=process_packet, timeout=20)

    print(f"Перехвачено пакетов: {len(packets)}")

    http_requests = [p for p in packets if p.haslayer(HTTPRequest)]
    http_responses = [p for p in packets if p.haslayer(HTTPResponse)]

    print(f"HTTP запросов: {len(http_requests)}")
    print(f"HTTP ответов: {len(http_responses)}")

    if len(packets) == 0:
        print("Трафик не обнаружен.")

except KeyboardInterrupt:
    print("Перехват прерван пользователем")
except Exception as e:
    print(f"Ошибка: {e}")

### Пример: отправка ICMP-пакета (ping)

In [ ]:
from scapy.all import IP, ICMP, sr1

# Создание ICMP пакета
packet = IP(dst="8.8.8.8")/ICMP()

# Отправка и получение ответа
reply = sr1(packet, timeout=2)

if reply:
    print(f"Ответ получен от {reply.src}")
    reply.show()
else:
    print("Таймаут - хост недоступен")

### Сканирование портов (базовый пример)

In [ ]:
from scapy.all import RandShort, TCP, sr1, IP

def scan_port(host, port):
    src_port = RandShort()
    packet = IP(dst=host)/TCP(sport=src_port, dport=port, flags="S")
    response = sr1(packet, timeout=1, verbose=0)

    if response:
        if response.haslayer(TCP):
            if response[TCP].flags == 18:  # SYN-ACK
                return True, "открыт"
            elif response[TCP].flags == 4:  # RST
                return False, "закрыт"
    return False, "фильтруется"


host = "127.0.0.1"
ports = [80, 443, 22, 21, 8080]

for port in ports:
    is_open, status = scan_port(host, port)
    print(f"Порт {port}: {status}")

### Безопасность и этика

**Важно для кибербезопасности:**

1. Используйте только на своих сетях или с разрешения.
2. Не перехватывайте чужой трафик без согласия.
3. Соблюдайте законы о защите данных.
4. Используйте для обучения и легального тестирования.

**Легальное использование:**
- Тестирование собственных приложений.
- Обучение и исследования.
- Пентесты с письменного разрешения.
- Анализ собственного трафика.

**Нелегальное использование:**
- Перехват чужого трафика без согласия.
- Атаки на чужие системы.
- Нарушение приватности.


## Ваши вопросы

## **Практика**



### Задача 1. Проверка доступности сайта

**Описание**

Напишите скрипт, который отправляет `GET`-запрос к заданному URL и выводит:

* код ответа (`status_code`);
* сообщение «Сайт доступен» или «Ошибка соединения».

**Пример:**

```python
Введите URL: https://example.com  
Ответ: 200 — Сайт доступен
```

---



### Задача 2. Получение заголовков ответа

**Описание**

Сделайте `GET`-запрос к сайту и выведите:

* все заголовки ответа (`response.headers`);
* значение заголовка `Content-Type`.


### Задача 3. Поиск ссылок на HTML-странице

**Описание**

Скачайте HTML-страницу и с помощью `BeautifulSoup`:

* найдите все теги `<a>`;
* выведите значения всех атрибутов `href`.

🔹 *Дополнительно*

Отфильтруйте только внешние ссылки (начинающиеся с `http`)


### Задача 4. Отправка POST-запроса с формой

**Описание**

Отправьте данные формы (`username`, `password`) методом `POST` на `https://httpbin.org/post`.

Выведите:

* JSON-ответ от сервера;
* переданные данные, как они отображаются на сервере.


### Задача 5. Извлечение email-адресов с веб-страницы

**Описание**

Загрузите HTML-страницу по указанному URL, найдите в тексте все email-адреса с помощью регулярных выражений и выведите их список.


## Ваши вопросы

## **Итоги**

Сегодня вы изучили:

✅ **HTTP-протокол** — фундамент веб-взаимодействия.  
✅ **requests** — классический инструмент для HTTP-запросов.  
✅ **httpx** — современный инструмент с асинхронностью для кибербезопасности.  
✅ **BeautifulSoup** — парсинг HTML для разведки и анализа.  
✅ **scapy** — анализ сетевых пакетов (базовые принципы).
✅ **Безопасность** — критические правила работы с HTTP.